# Exercises

The goal of this exercise is to analyse data taken by a radio receiver. You will study the statistical properties of thermal noise, and measure the noise temperature of some receiver. Since we are doing this remotely, I wrote a short code that generates mock data, but let's imagine together that it really does come from some commercially available receiver. Let's quickly look at one such receiver, the **AirSpy**, since this is the one we were using when this course was in person. A picture of the interior of the device is shown below.

<center><img src="Airspy_pic.png" /></center>

The AirSpy is a heterodyne receiver and can record up to $10^7$ samples per seconds, i.e. $\Delta t = 10^{-7}$ seconds, or 100 ns, which corresponds to a sampling rate of 10 MHz, and hence, a maximum bandwidth of 5 MHz. Thanks to its heterodyne mixer, it is tunable to any frequency between 24 and 1800 MHz. Below is a schematic of the relevant components that are relevant to the exercise.

<center><img src="schematic.png"></center>

- The **antenna** would typically be some sort of feed, such as a dipole or a horn, to couple the free space with the waveguide (cable). In this exercise, however, we will be connecting a terminating load (resistor) and vary its temperature to inspect the statistical properties of its noise, and measure the receiver temperature.
- The **low noise amplifier (LNA)** amplifies the signal coming from the antenna.
- The **mixer** multiplies the amplified signal with a pure tone generated by the **local oscillator (LO)**.
- The **mixer gain** amplifies the mixed signal in the band of interest.
- The **intermediate frequency (IF) filter** suppresses the power of the frequencies outside of the band of interest.
- The **IF gain** amplifies the filtered signal.
- The **analog-to-digital converter (ADC)** converts the analog signal to bits that are read by the computer.

## Collecting data

I wrote a function called **collect_data()**, which creates a time series with statistics and properties that mirror that of an AirSpy receiver connected to a terminating load. The function takes the following arguments:

- n_samples (int): determines the total number of time samples. I recommend using n_samples = int(1e6), which is also the default value of the function. More than that will be too slow, less than that is not enough for good statistics. If it's still too slow, one order of magnitude below, i.e. n_samples = int(1e5), still produces usable statistics. 
- T (float): the temperature of the load, in kelvins. I recommend trying with T = 300.0 (room temperature) first, which is also the default value of the function.

Note that this mock receiver has other properties which you have no control over, that I have determined. I call these properties the "metadata", and I define each of them below. Here they are:
- The sampling rate is 10 MHz ( $\Delta t = $ 100 ns), such that the  bandwidth is 5 MHz.
- The resolution is 12 bits, so the data range from 0 to 2¹² - 1 integers. **The bits are proportional to the measured voltage, so the power will be proportional to the bits squared.**

### **It takes approximately 3-5 seconds to collect the 1,000,000 samples, don't panic if it's not instantaneous!**

Here are some lines of codes to get you started.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from notebook_1_funcs import * # <-- This line is the most important, as it contains the collect_data() function!



# Here's some "metadata" that you will use later

delta_t = 100e-9 # in seconds
bandwidth = 1/(2*delta_t) # in Hz


# Those are the parameters of the data collection
n_samples = int(1e6)
T = 300 #in kelvins

data = collect_data(n_samples, T) # Will take approximately 3-5 seconds per each 1e6 samples

# Your data ranges from 0 to 2^12 - 1, but you want it to oscillate around 0, so subtract the mean
data = data - data.mean()

## Plot the time series

Plot the time series of the data you just collected. I recommend you don't plot the whole thing! A couple hundred samples should suffice. The y-axis will be in units of bits.

In [1]:
# Plot the time series here.

## Make a histogram of the data

Make a histogram of your samples (non-squared), and plot a Gaussian over it (hint: scipy.stats.pdf is a useful function to plot a gaussian). Use your samples' mean and variance to plot that Gaussian. Does the Gaussian line up with your histogram?

In [11]:
# Plot your histogram and gaussian here.

## Plot the power or system temperature over time

Plot the power, or system temperature, over time. Here, the power and the system temperature are equivalent because we do not know the constant of proportionality that would relate either of them to the units of bits squared. I recommend you plot all the data (not just a couple hundred samples), but by averaging every $k$ samples, with $k$ of the order 1000.

In [2]:
# Plot the power or system temperature here.

## Make a histogram of the power

Make histograms of your power data. Then, plot the histograms of sums of your data. For instance, sum every $k$ samples together, and make a histogram of the resulting sums. Try different $k$'s (e.g. 1,2,4,10,100), and compare it to the corresponding $\chi^2$ distribution (hint: scipy.statschi2 is a useful function to plot $\chi^2$ distributions). Do the $\chi^2$ distribution match up with your histograms? What happens to the distribution for large $k$'s?

Here are some additional hints: scale both axes logarithmically, for instance using ax.set_xscale('log') and similarly for yscale. If you do so, set the lower limit of the y-axis to 1 because the $\chi^2$ distribution is still defined below 1, which means in a log plot, there would be no lower limit, but your data is integer-valued so the range between 0 and 1 contains no useful information.

In [4]:
# Plot your histograms and chi^2 here.

## Check the radiometer equation in the time domain

Here is the radiometer equation again:

\begin{equation}
    \frac{\sigma_T}{T} = \sqrt{\frac{2}{k}} = \sqrt{\frac{1}{t\Delta \nu}}.
\end{equation}


In the theory notebook, we talk about the radiometer equation in the context of Fourier transforms. However, you can check that it holds even as we're still in the time domain. It may be simpler to use the $\sqrt{2/k}$ formulation, where $k$ is the number of time samples over which you do an average. You're encouraged to think about the $\sqrt{1/(t\Delta\nu)}$ formulation too.

Note also that it does not matter that we do not have the constant of proportionality to obtain the correct value of $T$ in kelvins, since we only need the *relative* uncertainty. That, we can readily obtain since the constants of proportionality will cancel out!

In [6]:
# Check the radiometer equation here. You don't have to plot anything.

## Plot a power spectrum in dB

If needed, double-check the definition of the power spectrum in the theory notebook, and plot it.
Here are some recommendations:
- Cut your data into smaller chunks of length $n$, and Fourier transform each of these chunks. Note that the result of these Fourier transforms will have length $n/2$, so you might want to choose the number of frequency bins you want, and have the length of the chunks be twice that number, to avoid having to do any re-binning.
- **Fast Fourier Transforms (FFTs)**, the most common numerical implementation of discrete Fourier transforms, are the fastest when they transform data with length of a power of 2. For the small data sets we are using here, it will make no noticeable difference, but it's good practice to always choose such lengths when possible. Hence, I recommend $n$=1024 or $n$=2048.
- Hint: Numpy's Fourier transform function (numpy.fft.fft()) may come in handy. It will produce outputs of lengths $n$ (instead of $n/2$), but half of the data is redundant. Read its documentation if you've never used it!
- Write a function that will chop up your data and do the Fourier transform, because you may need to do it again further down in the notebook.
- For a first try, just plot the mean spectrum over all your time chunks. You will get an idea of the shape of the output.
- In our context, the band we look at is arbitrary. In real life, you might want to avoid bands that are contaminated by **radio frequency intereference (RFI)**, e.g. cellphone signals. I chose 1200 MHz to be my intermediate frequency when preparing and testing this notebook, so that my band is 1200 MHz to 1205 MHz.

In [7]:
# Plot the power spectrum here.

## Check the radiometer equation relation in the frequency domain

Now that we are in the frequency domain, it makes sense to check the radiometer equation again. **For each frequency bin**, compute the ratio

\begin{equation}
    \frac{\sigma_T / T}{1/\sqrt{t\Delta \nu}}.
\end{equation}

Then, plot this ratio against the frequencies, and if the radiometer equation holds, it should be a flat line at 1. Try this by integrating over different number of time samples to get the power spectrum, and see what happens.

In [8]:
# Check the radiometer equation relation here.

## Waterfall plot of the data

Plot a waterfall plot of the data, i.e. a 2-D colormap where the x-axis is frequency, the y-axis is time, and the color is spectral power, in bits squared. Since we are using mock data, there should not be any notable feature throughout time.

In [10]:
# Plot the waterfall plot of the data here.

## Collect data at various load temperatures


We will look at what happens when we vary the temperature of the load. First, collect data at various temperatures. You are lucky, you simply have to change the "T" argument in the collect_data() function instead of submerging the resistor in boiling water! I recommend using temperature that range between 200 and 400, and using 3 to 5 different temperatures. It will take longer to collect the data than the first time around.

In [12]:
# Collect the data here.

Plot the system temperature of those data sets, with time as the x-axis, and the system temperature in bits squared as the y-axis.

In [13]:
# Plot the temperature of all your data sets here. I recommend you use a logged y-axis (ax.semilogy()).

Make histograms of the data for each of those temperatures. Make the histograms using the raw ADC values in bits (non-squared), to see the Gaussian and compare their shapes.

In [14]:
# Plot histograms for all four of those data sets.

## Spectrum vs load temperature

Generate the spectrum at the different temperatures you collected, and compare them (you can average the spectrum over all the samples).

In [15]:
# Plot the spectra here.

## Plot the power vs. load temperature, and do a linear fit

For our toy model, the system temperature can be expressed as

\begin{equation}
    T_{sys} = T_{load} + T_{receiver}.
\end{equation}

We want to know $T_{receiver}$. We know $T_{load}$, but we only know $T_{sys}$ up to some proportionality constant. In other words, we know $P_{sys} = k T_{sys}$, where $P_{sys}$ has units of bits squared, and $k$ is some unknown proportionality constant that converts a temperature in kelvins to a power in bits squared. We can thus write

\begin{equation}
    P_{sys} = k\left(T_{load} + T_{receiver}\right).
\end{equation}

Hence, if we plot $P_{sys}$ against $T_{load}$, the slope will be $k$, and the y-intercept divided by $k$ will be $T_{receiver}$! Now, you do this, using $T_{load}$ to be the input temperatures you used to collect data, and $P_{sys}$ to be the averaged power the band for each of your data sets.

In [16]:
# Plot the power vs load temperature.

## Receiver temperature vs spectral bin

Note that the temperature you just computed is the average over the whole band, but a good calibration would involve knowing the receiver temperature *per frequency*. This can be done with the exact same method, but going frequency-by-frequency with the power spectra you computed above.

In [17]:
# Plot the receiver temperature per frequency bin.